In [31]:
import rasterio as rio
from rasterio.plot import show
import numpy as np
from matplotlib import cm, pyplot as plt
from matplotlib.colors import LogNorm
import seaborn as sb
from datetime import datetime, timedelta

from ipyleaflet import Map, Marker
import folium
from sidecar import Sidecar
from PIL import Image

from ipyleaflet import Map, ImageOverlay, basemaps, basemap_to_tiles, WidgetControl
import os
from base64 import b64encode
from ipywidgets import IntSlider, SelectionSlider, VBox, Layout

In [32]:
data = rio.open('../R/2016-10-3_1000min.tif')
data_bounds = (data.bounds.left, data.bounds.right, \
          data.bounds.bottom, data.bounds.top)
bounds_arr = [[data.bounds.bottom, data.bounds.left], \
                  [data.bounds.top, data.bounds.right]]
bounds_arr

RasterioIOError: ../R/2016-10-3_1000min.tif: No such file or directory

In [ ]:
# transform pixel coordinates to lat long --> create heatmap to visualize with folium
rio.transform.xy(data.transform, 0, 0)

In [ ]:
def raster_to_heatmaps(raster_ts):
    ...

In [7]:
def raster_to_pngs(raster_data, t_start, t_delta=timedelta(minutes=15), colormap=cm.rainbow):
    
    img_arr = raster_data.read()
    
    print(np.nanmin(img_arr), np.nanmax(img_arr))
    
    img_arr[img_arr>2000] = np.nan
    
    img_arr[img_arr==0] = np.nan
    img_arr = np.log(img_arr)
    
    print(np.nanmin(img_arr), np.nanmax(img_arr))
    img_arr -= np.nanmean(img_arr)
    img_arr = (img_arr - np.nanmin(img_arr)) / (np.nanmax(img_arr) - np.nanmin(img_arr))
    #print(img_arr)
    
    x, y,z = np.where(np.isnan(img_arr))
    img_arr = colormap(img_arr, bytes = True)
    img_arr[x,y,z, -1] = 0
    
    print(img_arr.shape)
    
    #alpha = np.zeros(img_arr.shape)
    #alpha[np.where(np.isfinite(img_arr))] = 255
    #print(alpha.shape)
    
    #img_arr = np.uint8(colormap(img_arr)*255)
    
    #img_arr = np.concatenate((img_arr, alpha))
    
    os.makedirs('./png', exist_ok=True)
    
    t_range = np.array([t_start + t_delta*i for i in range(img_arr.shape[0])], dtype=datetime)
    
    for i in range(img_arr.shape[0]):
        im = Image.fromarray(img_arr[i])
        im.save(f'png/{t_range[i]}.png','PNG')
        
    
    #t_range = np.arange(t_start, datetime(2015,7,1), timedelta(days=1)).astype(datetime)
    return t_range

In [8]:
t_start = datetime(2016, 10, 3, 17)
t_range = raster_to_pngs(data, t_start)

0.0 inf


<ipython-input-7-1e0c660c17ee>:7: RuntimeWarning: invalid value encountered in greater
  img_arr[img_arr>2000] = np.nan


-5.2198973 7.600898
(67, 1304, 1244, 4)


In [6]:
str(t_range[0])

'2016-10-03 17:00:00'

In [ ]:
 = np.fft.fft2(img)
fshift = np.fft.fftshift(f)
magnitude_spectrum = 20*np.log(np.abs(fshift))

plt.subplot(121),plt.imshow(img, cmap = 'gray')
plt.title('Input Image'), plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(magnitude_spectrum, cmap = 'gray')
plt.title('Magnitude Spectrum'), plt.xticks([]), plt.yticks([])
plt.show()


In [6]:
data.meta

{'driver': 'GTiff',
 'dtype': 'float32',
 'nodata': None,
 'width': 1244,
 'height': 1304,
 'count': 9,
 'crs': CRS.from_epsg(4326),
 'transform': Affine(0.01, 0.0, -1.9407999999999994,
        0.0, -0.01, 57.94833862304688)}

In [30]:
1000/15

66.66666666666667

In [29]:
bounds=((data.bounds.bottom, data.bounds.left), \
                 (data.bounds.top, data.bounds.right))

center = ((bounds[0][0]+bounds[1][0])/2,(bounds[0][1]+bounds[1][1])/2)
m = Map(center=center, zoom=6)

#t_range = range(data.read().shape[0])

slider_layout = Layout(
    #display='flex',
    #flex_flow='row',
    #justify_content='space-between'
    #align_content = 'flex-end'
)

slider = SelectionSlider(description='time step:', 
                    options=t_range, 
                    orientation='horizontal',
                    layout={'width': '500px'}
)


widget_control = WidgetControl(widget=slider, position='topright', layout=form_item_layout)
m.add_control(widget_control)

io = None

def display_png(t):
    global io
    
    filename=f'png/{t}.png'
    with open(filename, 'rb') as f:
        d = b64encode(f.read())
    d = d.decode('ascii')
    p = 'data:image/png;base64,' + d

    if io is None:
        io = ImageOverlay(url=p, bounds=bounds, opacity=0.7)
        m.add_layer(io)
    else:
        #m.remove_layer(io)
        #io = ImageOverlay(url=p, bounds=bounds, opacity=0.6)
        #m.add_layer(io)
        io.url = p
        
def show_snapshot(change):
    display_png(change.new)
        

slider.observe(show_snapshot, 'value')
display_png(0)

sc = Sidecar(title='bird vis')
with sc:
    display(m)

Map(center=[51.42833862304688, 4.2792], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_ti…

In [153]:
center = [51.164878, 5.295410]

img = data.read()
#img = np.random.rand(*img.shape)
img[img==0] = np.nan    
img = np.log(img)
img = (img - np.nanmin(img)) / (np.nanmax(img) - np.nanmin(img))

@widgets.interact(
    t=(0, data.read().shape[0]-1))
def plot(t=0):

    fig, ax = plt.subplots(figsize=(15,15))
    #ax = plt.axes(projection=ccrs.Mercator(central_longitude=center[0], min_latitude=data.bounds.bottom, max_latitude=data.bounds.top), extent=bounds)
    #ax.coastlines()
    
    m = ax.imshow(img[t], extent=data_bounds, interpolation='none')
    #ax.set(xlabel='longitude', ylabel='latitude')
    plt.colorbar(m)
    
    plt.show()

interactive(children=(IntSlider(value=0, description='t', max=8), Output()), _dom_classes=('widget-interact',)…

In [70]:
center = [51.164878, 5.295410]
m = folium.Map(location=center, tiles = 'Stamen Terrain', zoom_start = 6)

img = data.read()
img[img==0] = np.nan
#img = np.nan_to_num(img)
img = np.log(img)
img = (img - np.nanmin(img)) / (np.nanmax(img) - np.nanmin(img))

sc = Sidecar(title='Sidecar Output')
time = 0
with sc:
    #@widgets.interact(time=(0, data.read().shape[0]-1))
    #def raster_overlay(time):
        #img = np.log(img)
        #img = np.random.rand(*img.shape)
        
    folium.raster_layers.ImageOverlay(plt.cm.get_cmap('rainbow')(img[time]), opacity=0.6, bounds=bounds_arr, mercator_project=True).add_to(m)
    display(m)

In [3]:
sc = Sidecar()
with sc:
    print('test')

test
